<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej"
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

In [ ]:
import random
import numpy as np
from torchvision import datasets, transforms

In [ ]:
!wget -O mnist.npz https://s3.amazonaws.com/img-datasets/mnist.npz

--2023-11-06 22:43:04--  https://s3.amazonaws.com/img-datasets/mnist.npz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.44.184, 52.216.216.112, 52.216.138.61, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.44.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11490434 (11M) [application/octet-stream]
Saving to: ‘mnist.npz’

mnist.npz           100%[===================>]  10.96M  37.0MB/s    in 0.3s    

2023-11-06 22:43:04 (37.0 MB/s) - ‘mnist.npz’ saved [11490434/11490434]



In [ ]:
# Let's read the mnist dataset

def load_mnist(path='mnist.npz'):
    with np.load(path) as f:
        x_train, _y_train = f['x_train'], f['y_train']
        x_test, _y_test = f['x_test'], f['y_test']

    x_train = x_train.reshape(-1, 28 * 28) / 255.
    x_test = x_test.reshape(-1, 28 * 28) / 255.

    y_train = np.zeros((_y_train.shape[0], 10))
    y_train[np.arange(_y_train.shape[0]), _y_train] = 1

    y_test = np.zeros((_y_test.shape[0], 10))
    y_test[np.arange(_y_test.shape[0]), _y_test] = 1

    return (x_train, y_train), (x_test, y_test)

(x_train, y_train), (x_test, y_test) = load_mnist()

## Exercise 1

In this exercise your task is to fill in the gaps in this code by implementing the backpropagation algorithm
Once this is done, you can run the network on the MNIST example and see how it performs. Feel free to play with the parameters. Your model should achieve 90%+ accuracy after a few epochs.


## Exercise 2 (Optional)

Implement a "fully vectorized" version, i.e. one using matrix operations instead of going over examples one by one within a minibatch.


In [ ]:
# Exercise 1

def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    # Derivative of the sigmoid
    return sigmoid(z)*(1-sigmoid(z))

class Network(object):
    def __init__(self, sizes):
        # initialize biases and weights with random normal distr.
        # weights are indexed by target node first
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
    def feedforward(self, a):
        # Run the network on a single case
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def update_mini_batch(self, x_mini_batch, y_mini_batch, eta):
        # Update networks weights and biases by applying a single step
        # of gradient descent using backpropagation to compute the gradient.
        # The gradient is computed for a mini_batch.
        # eta is the learning rate
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in zip(x_mini_batch, y_mini_batch):
            delta_nabla_b, delta_nabla_w = self.backprop(x.reshape(784,1), y.reshape(10,1))
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(x_mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(x_mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        # For a single input (x,y) return a tuple of lists.
        # First initialize the list of gradient arrays
        delta_nabla_b = [np.zeros(b.shape) for b in self.biases] # Gradients over biases
        delta_nabla_w = [np.zeros(w.shape) for w in self.weights] # Gradients over weights

        # Feedforward - store all activations and weighted inputs

        # Values before activation function, layer by layer, shapes (L_1), ..., (10).
        zs = [] # z_i is the z_{i-1} * w_i
        # Values after activation function (including inputs to the first layer), shapes (784), (L_1), ..., (10).
        activations = [] # a_i is z_i with applied activation function
        activations.append(x) # we add our input matrix to activations
        i = 1

        # here we just add to empty array values that are initially in the neurons of neural network
        for w, b, a in zip(self.weights, self.biases, activations):
            z = np.dot(w, activations[i-1]) + b # since values on whole layer is just multiplying matrix of weights with matrix of zs and to each we add bias[it]
            zs.append(z)
            activations.append(sigmoid(z))
            i = i + 1

        # Now go backward from the final cost applying backpropagation
        # It’s a process of fine-tuning the weights of a network based on
        # the errors or loss obtained in the previous epoch (iteration).
        dLdg = self.cost_derivative(activations[-1], y)
        dLdfs = []

        # We are going backwards, not counting the first "layer of activations", since it's just the input
        for W, g in zip(reversed(self.weights), reversed(activations[1:])):

          # Formula for dL over df from the lecture.
          dLdf = np.multiply(dLdg, np.multiply(g, (1 - g)))

          # We are adding dL / df to the front list, but remember we are going backwards in our Neural Network.
          dLdfs.append(dLdf)

          # Formula from dL over dg from the lecture.
          dLdg = np.matmul(W.T, dLdf)

        # We are updating our deltas for each weight using matrices that we calculated in backprop
        delta_nabla_w = [dLdf @ g.T for dLdf, g in zip(reversed(dLdfs), activations[:-1])]
        # delta_nabla_b = [np.sum(dLdf, axis=1).reshape(dLdf.shape[0], 1) for dLdf in reversed(dLdfs)]

        return (delta_nabla_b, delta_nabla_w)


    def evaluate(self, x_test_data, y_test_data):
        # Count the number of correct answers for test_data
        test_results = [(np.argmax(self.feedforward(x_test_data[i].reshape(784,1))), np.argmax(y_test_data[i]))
                        for i in range(len(x_test_data))]
        # return accuracy
        return np.mean([int(x == y) for (x, y) in test_results])

    def cost_derivative(self, output_activations, y):
        return (output_activations-y)

    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        x_train, y_train = training_data
        if test_d5ata:
            x_test, y_test = test_data
        for j in range(epochs):
            for i in range(x_train.shape[0] // mini_batch_size):
                x_mini_batch = x_train[i*mini_batch_size:(i*mini_batch_size + mini_batch_size)]
                y_mini_batch = y_train[i*mini_batch_size:(i*mini_batch_size + mini_batch_size)]
                self.update_mini_batch(x_mini_batch, y_mini_batch, eta)
            if test_data:
                print("Epoch: {0}, Accuracy: {1}".format(j, self.evaluate(x_test, y_test)))
            else:
                print("Epoch: {0}".format(j))


network = Network([784,30,10])
network.SGD((x_train, y_train), epochs=50, mini_batch_size=100, eta=3., test_data=(x_test, y_test))



Epoch: 0, Accuracy: 0.7327
Epoch: 1, Accuracy: 0.8513
Epoch: 2, Accuracy: 0.881
Epoch: 3, Accuracy: 0.8948
Epoch: 4, Accuracy: 0.9047
Epoch: 5, Accuracy: 0.9086
Epoch: 6, Accuracy: 0.9138
Epoch: 7, Accuracy: 0.9164
Epoch: 8, Accuracy: 0.9193
Epoch: 9, Accuracy: 0.9217
Epoch: 10, Accuracy: 0.9234
Epoch: 11, Accuracy: 0.924
Epoch: 12, Accuracy: 0.9262
Epoch: 13, Accuracy: 0.9276
Epoch: 14, Accuracy: 0.9282
Epoch: 15, Accuracy: 0.929
Epoch: 16, Accuracy: 0.9299
Epoch: 17, Accuracy: 0.9306
Epoch: 18, Accuracy: 0.9321
Epoch: 19, Accuracy: 0.9331
Epoch: 20, Accuracy: 0.9339
Epoch: 21, Accuracy: 0.935
Epoch: 22, Accuracy: 0.9354
Epoch: 23, Accuracy: 0.9359
Epoch: 24, Accuracy: 0.9368
Epoch: 25, Accuracy: 0.937
Epoch: 26, Accuracy: 0.9379
Epoch: 27, Accuracy: 0.9377
Epoch: 28, Accuracy: 0.9378
Epoch: 29, Accuracy: 0.9377
Epoch: 30, Accuracy: 0.9379
Epoch: 31, Accuracy: 0.9383
Epoch: 32, Accuracy: 0.9379
Epoch: 33, Accuracy: 0.9387
Epoch: 34, Accuracy: 0.939
Epoch: 35, Accuracy: 0.9393
Epoch: 3